In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import datetime
import csv
import os
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from modules.nypd_data import read_orig_file
from modules.nypd_data import filter_raw_data
from modules.nypd_data import save_dated_felonies
from modules.nypd_data import load_dated_felonies
from modules.nypd_data import save_clean_felonies
from modules.nypd_data import load_clean_felonies
from modules.nypd_data import add_offense_category
from modules.nypd_data import add_datetime_columns
from modules.nypd_data import pivot_felonies
from modules.nypd_data import save_pivoted_felonies
from modules.nypd_data import load_pivoted_felonies

In [3]:
from modules.prediction import load_all_data
from modules.prediction import precrime_train_test_split, create_test_period, precrime_train_test_split
from modules.prediction import split_into_X_y
from modules.prediction import load_splits
from modules.prediction import create_all_splits
from modules.prediction import sample_model
from modules.poly_ridge import poly_ridge_model
from modules.fancy_time_series import fancy_time_series_model
from modules.eval_model import eval_predictions


In [4]:
from modules.nypd_data import load_clean_felonies
from modules.nyc_shapefiles import read_nyc_shapefiles
from modules.weather import get_precinct_centroids
from modules.weather import get_mean_latlon
from modules.weather import write_weather_data
from modules.weather import read_weather_data
from modules.weather import read_api_key

In [23]:
# Generating file for 2017
# Get the ytd crime file from the open data website and 
# store it in the folder precrime_data/raw
# save_dated_felonies('../precrime_data/raw/'
#                     , 'NYPD_Complaint_Data_Current_YTD.csv'
#                    , '../precrime_data/dated_felonies_2017.csv')
# dated_felonies = load_dated_felonies('../precrime_data/'
#                                      , 'dated_felonies_2017.csv', datetime='2017-01-01 00:00:00')
# save_clean_felonies('../precrime_data/', 'dated_felonies_2017.csv'
#                    , '../precrime_data/clean_felonies_2017.csv')

# nypd_data_2017 = load_clean_felonies('../precrime_data/'
#                                      , 'clean_felonies_2017.csv')
# add_offense_category(nypd_data_2017)
# add_datetime_columns(nypd_data_2017)
# pivoted_2017 = pivot_felonies(nypd_data_2017)
# save_pivoted_felonies(nypd_data_2017, data_path='../precrime_data/'
#                       , pivot_file='pivoted_felonies_2017.csv')

    

In [24]:
# Creating weather data until 2017-11-30 

# Before running: execute "tar -xvf weather_hist.tar.gz"
# from the root directory of the git repo
#
# To make the files from scratch (not necessary):
#
#nypd_data = load_clean_felonies()
#precinct_dict, tract_dict, merged_census_info = read_nyc_shapefiles()
#centroids = get_precinct_centroids(precinct_dict)
#nyc_mean_latlon = get_mean_latlon(centroids)
#nypd_dates = nypd_data['COMPLAINT_DATETIME'].dt.date.unique()
#hours = range(2,26,4)
#darksky_api_key = read_api_key()

In [25]:
#new_nypd_dates = []
#one_day = datetime.timedelta(days=1)
#cur_day = nypd_dates[-1] + one_day
#while cur_day.month < 12:
#    new_nypd_dates.append(cur_day)
#    cur_day += one_day

In [26]:
#write_weather_data(new_nypd_dates, hours, darksky_api_key, nyc_mean_latlon, append_output=True)

In [27]:
# import traceback
# import warnings
# import sys

# def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

#     log = file if hasattr(file,'write') else sys.stderr
#     traceback.print_stack(file=log)
#     log.write(warnings.formatwarning(message, category, filename, lineno, line))

# warnings.showwarning = warn_with_traceback

In [5]:
crime_data_train = load_all_data()

crime_data_test = load_all_data(data_path='../precrime_data/'
                           , pivot_file='pivoted_felonies_2017.csv' )

crime_data = pd.concat([crime_data_train, crime_data_test])


C:\Users\ngamf\Anaconda3\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
str_date = datetime.datetime.strptime('2017-01-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2017-07-01', '%Y-%m-%d')
test_times = create_test_period(str_date,end_date)
X_train, X_test, y_train, y_test = precrime_train_test_split(crime_data, test_times)

In [7]:
y_poly = poly_ridge_model(X_train, y_train, X_test)




In [8]:
y_ts = fancy_time_series_model(X_train, y_train, X_test, y_test)

y_hybrid = (y_poly + y_ts) / 2

In [9]:
eval_predictions(X_test, y_test, y_hybrid)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -0.6, RMSE =     0.039, RMSE (%) =  2827.521
Rape:             R2 =     -0.3, RMSE =     0.107, RMSE (%) =   963.968
Robbery:          R2 =      2.8, RMSE =     0.288, RMSE (%) =   366.856
FelonyAssault:    R2 =      3.5, RMSE =     0.377, RMSE (%) =   328.765
Burglary:         R2 =      0.8, RMSE =     0.267, RMSE (%) =   399.288
GrandLarceny:     R2 =      8.2, RMSE =     0.493, RMSE (%) =   222.044
GrandLarcenyAuto: R2 =      0.8, RMSE =     0.173, RMSE (%) =   592.334
Fraud:            R2 =      1.5, RMSE =     0.151, RMSE (%) =   683.975
Forgery:          R2 =      2.5, RMSE =     0.183, RMSE (%) =   576.411
Arson:            R2 =     -0.5, RMSE =     0.064, RMSE (%) =  1639.196
Drugs:            R2 =      2.0, RMSE =     0.167, RMSE (%) =   624.612
Weapons:          R2 =      1.9, RMSE =     0.174, RMSE

In [11]:
crime_data_js = load_all_data(data_path='../precrime_data/'
                           , pivot_file='new_pivoted_felonies.csv')

C:\Users\ngamf\Anaconda3\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
from modules.prediction import create_test_period
split_all_2017 = create_test_period(datetime.date(2017,1,1),datetime.date(2017,10,1))
split_thru_june_2017 = create_test_period(datetime.date(2017,1,1),datetime.date(2017,7,1))

In [14]:
train_test_data = create_all_splits(crime_data_js, {'all_2017': split_all_2017, 'thru_june': split_thru_june_2017})

In [15]:
X_train_2017, X_test_2017, y_train_2017, y_test_2017 = train_test_data['all_2017']
X_train_june, X_test_june, y_train_june, y_test_june = train_test_data['thru_june']

In [16]:
y_poly_2017 = poly_ridge_model(X_train_2017, y_train_2017, X_test_2017)
y_poly_june = poly_ridge_model(X_train_june, y_train_june, X_test_june)

y_ts_2017 = fancy_time_series_model(X_train_2017, y_train_2017, X_test_2017, y_test_2017)
y_ts_june = fancy_time_series_model(X_train_june, y_train_june, X_test_june, y_test_june)

y_hybrid_2017 = (y_poly_2017 + y_ts_2017) / 2
y_hybrid_june = (y_poly_june + y_ts_june) / 2

In [17]:
eval_predictions(X_test_2017, y_test_2017, y_hybrid_2017)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -0.4, RMSE =     0.041, RMSE (%) =  2730.391
Rape:             R2 =      0.0, RMSE =     0.112, RMSE (%) =   918.150
Robbery:          R2 =      3.7, RMSE =     0.291, RMSE (%) =   358.580
FelonyAssault:    R2 =      5.4, RMSE =     0.382, RMSE (%) =   318.653
Burglary:         R2 =      1.0, RMSE =     0.269, RMSE (%) =   391.755
GrandLarceny:     R2 =     10.7, RMSE =     0.501, RMSE (%) =   213.885
GrandLarcenyAuto: R2 =      1.4, RMSE =     0.183, RMSE (%) =   562.755
Fraud:            R2 =      1.7, RMSE =     0.152, RMSE (%) =   675.712
Forgery:          R2 =      2.6, RMSE =     0.181, RMSE (%) =   586.003
Arson:            R2 =     -0.4, RMSE =     0.065, RMSE (%) =  1623.359
Drugs:            R2 =      2.8, RMSE =     0.168, RMSE (%) =   623.468
Weapons:          R2 =      3.2, RMSE =     0.174, RMSE

In [18]:
eval_predictions(X_test_june, y_test_june, y_hybrid_june)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -0.5, RMSE =     0.040, RMSE (%) =  2794.489
Rape:             R2 =      0.1, RMSE =     0.111, RMSE (%) =   915.342
Robbery:          R2 =      3.8, RMSE =     0.288, RMSE (%) =   363.713
FelonyAssault:    R2 =      5.3, RMSE =     0.377, RMSE (%) =   322.523
Burglary:         R2 =      1.2, RMSE =     0.269, RMSE (%) =   394.671
GrandLarceny:     R2 =     11.6, RMSE =     0.499, RMSE (%) =   211.911
GrandLarcenyAuto: R2 =      1.2, RMSE =     0.174, RMSE (%) =   586.598
Fraud:            R2 =      1.9, RMSE =     0.160, RMSE (%) =   643.028
Forgery:          R2 =      3.1, RMSE =     0.185, RMSE (%) =   564.507
Arson:            R2 =     -0.4, RMSE =     0.066, RMSE (%) =  1578.134
Drugs:            R2 =      2.7, RMSE =     0.167, RMSE (%) =   620.130
Weapons:          R2 =      2.9, RMSE =     0.173, RMSE

In [19]:
eval_predictions(X_test_june, y_test_june, y_poly_june)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =      0.0, RMSE =     0.040, RMSE (%) =  2787.736
Rape:             R2 =      0.4, RMSE =     0.111, RMSE (%) =   913.859
Robbery:          R2 =      3.3, RMSE =     0.288, RMSE (%) =   364.645
FelonyAssault:    R2 =      4.9, RMSE =     0.378, RMSE (%) =   323.177
Burglary:         R2 =      0.4, RMSE =     0.270, RMSE (%) =   396.294
GrandLarceny:     R2 =      9.2, RMSE =     0.506, RMSE (%) =   214.827
GrandLarcenyAuto: R2 =      0.7, RMSE =     0.175, RMSE (%) =   588.015
Fraud:            R2 =      1.7, RMSE =     0.160, RMSE (%) =   643.468
Forgery:          R2 =      1.9, RMSE =     0.187, RMSE (%) =   568.051
Arson:            R2 =      0.0, RMSE =     0.066, RMSE (%) =  1574.851
Drugs:            R2 =      1.2, RMSE =     0.168, RMSE (%) =   624.768
Weapons:          R2 =      2.3, RMSE =     0.173, RMSE

In [20]:
eval_predictions(X_test_june, y_test_june, y_ts_june)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -2.0, RMSE =     0.040, RMSE (%) =  2815.388
Rape:             R2 =     -1.2, RMSE =     0.112, RMSE (%) =   921.286
Robbery:          R2 =      2.8, RMSE =     0.289, RMSE (%) =   365.595
FelonyAssault:    R2 =      4.3, RMSE =     0.379, RMSE (%) =   324.184
Burglary:         R2 =      0.3, RMSE =     0.270, RMSE (%) =   396.455
GrandLarceny:     R2 =     11.5, RMSE =     0.500, RMSE (%) =   212.093
GrandLarcenyAuto: R2 =      0.1, RMSE =     0.175, RMSE (%) =   589.811
Fraud:            R2 =      0.8, RMSE =     0.161, RMSE (%) =   646.612
Forgery:          R2 =      1.9, RMSE =     0.187, RMSE (%) =   567.885
Arson:            R2 =     -1.8, RMSE =     0.067, RMSE (%) =  1589.589
Drugs:            R2 =      2.0, RMSE =     0.168, RMSE (%) =   622.085
Weapons:          R2 =      1.6, RMSE =     0.174, RMSE